# Predicting Apartment Prices in Mexico City

In this project, I will predict the prices of apartments in Mexico City (Distrito Federal)

In [23]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

## Data Import, Cleaning and Preparation

There are 5 different csv files for this project. I will import one of them, clean it, add all the cleaning steps to a function, and apply the function to the rest of the csv files.

In [24]:
# Import csv file and inspect it
df1 = pd.read_csv('/workspaces/Predicting-Apartment-Prices-in-Mexico-City/mexico-city-real-estate-1.csv')
display(df1.head())
display(df1.info())

,Unnamed: 0,operation,property_type,place_with_parent_names,lat-lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url
0,1,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,"23.634501,-102.552788",5500000.0,MXN,5450245.50,289775.66,NaN,54.0,NaN,101851.851900,NaN,NaN,NaN,http://miguel-hidalgo-df.properati.com.mx/o3zb...
1,2,sell,house,|Iztapalapa|Distrito Federal|México|,"19.31033,-99.068557",1512000.0,MXN,1498321.97,79661.96,NaN,80.0,NaN,18900.000000,NaN,NaN,NaN,http://iztapalapa.properati.com.mx/q7t0_venta_...
2,3,sell,apartment,|Tlalpan|Distrito Federal|México|,"19.279771,-99.234597",926667.0,MXN,918284.00,48822.82,NaN,100.0,NaN,9266.670000,NaN,NaN,NaN,http://tlalpan.properati.com.mx/qbi4_venta_dep...
3,4,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,"23.634501,-102.552788",6410000.0,MXN,6352013.39,337720.36,NaN,135.0,NaN,47481.481480,NaN,NaN,NaN,http://miguel-hidalgo-df.properati.com.mx/opeq...
4,5,sell,apartment,|Benito Juárez|Quintana Roo|México|,"21.1902642,-86.8198375",875000.0,USD,16457437.50,875000.00,0.0,263.0,NaN,3326.996198,NaN,NaN,NaN,http://cancun.properati.com.mx/hg4t_venta_depa...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4628 entries, 0 to 4627
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  4628 non-null   int64  
 1   operation                   4628 non-null   object 
 2   property_type               4628 non-null   object 
 3   place_with_parent_names     4628 non-null   object 
 4   lat-lon                     4144 non-null   object 
 5   price                       4538 non-null   float64
 6   currency                    4538 non-null   object 
 7   price_aprox_local_currency  4538 non-null   float64
 8   price_aprox_usd             4538 non-null   float64
 9   surface_total_in_m2         1668 non-null   float64
 10  surface_covered_in_m2       4436 non-null   float64
 11  price_usd_per_m2            1150 non-null   float64
 12  price_per_m2                4249 non-null   float64
 13  floor                       291 n

None

Some of the columns contain null values. I will drop columns where at least 50% of its data are null values. I will impute the values in the remaining columns.

In [25]:
# Checking columns with 50% of more missing values
(df1.isna().sum() / len(df1)) >= 0.5

Unnamed: 0                    False
operation                     False
property_type                 False
place_with_parent_names       False
lat-lon                       False
price                         False
currency                      False
price_aprox_local_currency    False
price_aprox_usd               False
surface_total_in_m2            True
surface_covered_in_m2         False
price_usd_per_m2               True
price_per_m2                  False
floor                          True
rooms                          True
expenses                       True
properati_url                 False
dtype: bool

The columns which have 50% of more missing data are: `surface_total_in_m2`, `price_usd_per_m2`, `floor`, `rooms`, `expenses`.
I will drop these columns

In [26]:
# Drop affected columns
df1.drop(columns=['surface_total_in_m2',
                  'price_usd_per_m2',
                  'floor',
                  'rooms',
                  'expenses'],
                  inplace=True)

Since I'm concerned with only the apartments in Mexico City - Distrito Federal, I will subset on the `property_type` and filter for apartments.

Also, I will subset on `place_with_parent_names` and filter for those that contain Distrito Federal

In [27]:
# Subset on properties that are apartments
apt_filter = df1['property_type'] == 'apartment'

# Subset on locations that contain Distrito Federal
state_filter = df1['place_with_parent_names'].str.contains('Distrito Federal')

# Apply the filters
df1 = df1[apt_filter & state_filter]

I will split the `lat-lon` column into seperate `lat` and `lon` columns. I will then drop the `lat-lon` column afterwards.

In [28]:
# Create seperate lat and lon columns and convert to float data type
df1[['lat', 'lon']] = (
    df1['lat-lon'].str.split(',', expand=True)
    .astype(float)
)

# Drop lat-lon column
df1.drop(
    columns=['lat-lon'],
    inplace=True
)

Mexico City is divided into boroughs. The `place_with_parent_names` contains the location hierarchy of each property in this order: |Borough|State|Country|.

I will create a new column, `borough`, to which I will extract the borough name from the location hierarchy and drop the column containing the location hierarchy.

In [29]:
# Extract borough names
df1['borough'] = (
    df1['place_with_parent_names'].str
    .split('|', expand=True)[1]
    )

# Drop place_with_parent_names column
df1.drop(
    columns=['place_with_parent_names'],
    inplace=True
)

I will inspect the numerical columns, particularly the `price_aprox_usd` and `surface_covered_in_m2`.

In [30]:
# Inspect price_aprox_usd and surface_covered_in_m2 columns
df1[['price_aprox_usd', 'surface_covered_in_m2']].describe()

,price_aprox_usd,surface_covered_in_m2
count,2.754000e+03,2624.000000
mean,2.154891e+05,128.730945
std,3.972478e+05,656.209935
min,7.823940e+03,0.000000
25%,5.133285e+04,60.000000
50%,9.620551e+04,80.000000
75%,2.046332e+05,110.000000
max,8.429837e+06,31840.000000


The price column shows a mean of over $200k, while the median is under $100k, indicating the presence of a few apartments with exceptionally high prices. This is highlighted by the maximum price exceeding $8 million.

For the area column, the minimum value of 0 square meters is unrealistic for an apartment, while the maximum area of over 30,000 square meters is significantly larger than the 75th percentile value of 110 square meters.

To address these outlier issues, I will filter for apartments priced at $150k or less and those with areas between the 10th and 90th percentiles.

In [31]:
# Defining the filters to drop outliers
price_filter = df1['price_aprox_usd'] <= 150000

area_low, area_high = df1['surface_covered_in_m2'].quantile([.1, .9])
area_filter = df1['surface_covered_in_m2'].between(area_low, area_high)

# Applying the filters
df1 = df1[price_filter & area_filter]

The next thing I will look at are categorical columns with low or high cardinality. Low cardinality columns are those with few unique values while the high cardinality ones with many unique values.

If there's only one category in a column, it won't provide any unique information to the model. Columns where nearly every row has its own category won't help the model in identifying useful trends in the data.

I will calculate the number of unique values for the categorical columns in the data.

In [32]:
# Number of unique values in each categorical columns
df1.select_dtypes('object').nunique()

operation           1
property_type       1
currency            1
properati_url    1508
borough            15
dtype: int64

Low cardinality columns: `operation`, `property_type`, `currency`

High cardinality columns: `properati_url`

I will drop these columns

In [40]:
# Drop low and high cardinality columns
df1.drop(
    columns=[
    'operation',
    'property_type',
    'currency',
    'properati_url'
    ],
    inplace=True
)

It is also important to drop any columns that will constitute leakage, that is, features that were created using our target (price) which would give it access to information it's not supposed to.

Example of such columns are: `price`, `price_aprox_local_currency`, `price_per_m2`, `price_usd_per_m2`(already dropped with the null values earlier)

There is also the issue of multi-colinearity. Multicollinearity happens when two or more features in a dataset are too closely related. This makes it hard for the model to know which one matters more. Example of such columns are: `surface_area_total_in_m2` and  `rooms`, as they will be positively correlated with `surface_covered_in_m2`.

Though these columns were aleady dropped with the null values earlier.



In [46]:
# Drop leaky columns
df1.drop(
    columns = [
        'price',
        'price_aprox_local_currency',
        'price_per_m2'
    ],
    inplace=True
)

There is still one column that should be dropped as it is not necessary for this project: `Unnamed: 0`.

In [49]:
# Drop unnecessary column
df1.drop(columns=['Unnamed: 0'], inplace=True)

Now that I'm done preprocessing the data, I will rename the columns appropriately

In [51]:
# Rename the remaining columns appropriately
rename_dict = {
    'price_aprox_usd': 'price_usd',
    'surface_covered_in_m2': 'surface_area_m2',
    'lat': 'lat',
    'lon': 'lon',
    'borough': 'borough'
}

df1.rename(columns= rename_dict, inplace= True)

To apply these preprocessing steps on the other csv files, I will build a `wrangle` function which will include all those steps.

In [54]:
# Defining the wrangle function
def wrangle(filepath):
    # Read in data
    df = pd.read_csv(filepath)

    # Drop columns with >= 50% nulls
    df.drop(columns=['surface_total_in_m2',
                  'price_usd_per_m2',
                  'floor',
                  'rooms',
                  'expenses'
                  ],
            inplace=True)
    
    # Subset on properties that are apartments
    apt_filter = df['property_type'] == 'apartment'

    # Subset on locations that contain Distrito Federal
    state_filter = df['place_with_parent_names'].str.contains('Distrito Federal')

    # Apply the filters
    df = df[apt_filter & state_filter]

    # Create seperate lat and lon columns and convert to float data type
    df[['lat', 'lon']] = (
        df['lat-lon'].str.split(',', expand=True)
        .astype(float)
        )
    # Drop lat-lon column
    df.drop(
        columns=['lat-lon'],
        inplace=True
        )
    
    # Extract borough names
    df['borough'] = (
        df['place_with_parent_names'].str
        .split('|', expand=True)[1]
        )
    # Drop place_with_parent_names column
    df.drop(
        columns=['place_with_parent_names'],
        inplace=True
        )
    # Defining the filters to drop outliers
    price_filter = df['price_aprox_usd'] <= 150000

    area_low, area_high = df['surface_covered_in_m2'].quantile([.1, .9])
    area_filter = df['surface_covered_in_m2'].between(area_low, area_high)

    # Applying the filters
    df = df[price_filter & area_filter]
    
    # Drop low and high cardinality columns
    df.drop(
        columns=[
            'operation',
            'property_type',
            'currency',
            'properati_url'
            ],
        inplace=True
        )
    
    # Drop leaky columns
    df.drop(
        columns = [
            'price',
            'price_aprox_local_currency',
            'price_per_m2'
            ],
        inplace=True
        )
    
    # Drop unnecessary column
    df.drop(columns=['Unnamed: 0'], inplace=True)

    # Rename the remaining columns appropriately
    rename_dict = {
        'price_aprox_usd': 'price_usd',
        'surface_covered_in_m2': 'surface_area_m2',
        'lat': 'lat',
        'lon': 'lon',
        'borough': 'borough'
        }
    df.rename(columns= rename_dict, inplace= True)

In [52]:
df1.columns

Index(['price_usd', 'surface_area_m2', 'lat', 'lon', 'borough'], dtype='object')